In [1]:
import os
from tqdm.auto import tqdm
import random
from PIL import Image

In [2]:
def filter_data(data, len_range, max_space):
    if data["label"] is None:
        return False
    if len(data["label"]) < len_range[0] or len(data["label"]) > len_range[1]:
        return False
    if data["label"].count(' ') > max_space:
        return False
    if data["label"].isascii() == False:
        return False    
    for char in "\n\r\xad\xa0":
        if char in data["label"]:
            return False
    try:
        image = Image.open(data["image_path"])
        image.verify()
        if image.format != "JPEG" and image.format != "PNG":
            print("not a valid format")
            return False
    except:
        print("invalid image or path")
        return False

    # if os.path.exists(data["image_path"]) == False:
    #     return False
    return True

def generate_data_file(
        data_file_dir,
        multi_fonts_dir,
        mjsynth_dir,
        max_multi_fonts_len,
        max_mjsynth_len,
        len_range,
        max_space
):
    dataset = []
    data_file = open(os.path.join(data_file_dir, "data.txt"), "a+")
    multi_fonts_data = open(os.path.join(multi_fonts_dir, 'data.txt')).readlines()
    mjsynth_data = open(os.path.join(mjsynth_dir, "imlist.txt")).readlines()
    random.shuffle(multi_fonts_data)
    random.shuffle(mjsynth_data)
    multi_fonts_data = multi_fonts_data[:max_multi_fonts_len]
    mjsynth_data = mjsynth_data[:max_mjsynth_len]
    for line in tqdm(multi_fonts_data):
        splitted_line = line.split(' ', 1)
        label = splitted_line[1].split('\n')[0]
        dataset.append({"image_path": os.path.join(multi_fonts_dir, splitted_line[0]), "label": label})
    for image_name in tqdm(mjsynth_data):
        image_name = image_name.split('\n')[0]
        label = image_name.split('/')[-1].split('_')[1]
        dataset.append({"image_path": os.path.join(mjsynth_dir, image_name), "label": label})
    random.shuffle(dataset)
    dataset = list(filter(lambda data: filter_data(data, len_range, max_space), tqdm(dataset)))
    for data in tqdm(dataset):
        data_file.write(f"{data['image_path']} {data['label']}\n")
    data_file.close()
generate_data_file(
    "./",
    "../../datasets/multi-fonts-generated-text/",
    "../../datasets/mjsynth/mnt/ramdisk/max/90kDICT32px/",
    500_000,
    500_000,
    len_range=(3, 32),
    max_space=3
)

  0%|          | 0/500000 [00:00<?, ?it/s]

  0%|          | 0/500000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/996791 [00:00<?, ?it/s]